# Q1. Encrypt and Decrypt Emails Using PGP.

In [3]:
import random
import string
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding as sym_padding

# Generate RSA key pair (public and private keys)
def generate_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()

    # Serialize and return the keys
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    return private_pem, public_pem

# Function to generate a random AES key (symmetric key)
def generate_aes_key(length=32):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length)).encode('utf-8')

# Encrypt the message using the AES key
def encrypt_email_aes(aes_key, message):
    # Pad message for AES block size
    padder = sym_padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(message.encode()) + padder.finalize()

    # Encrypt the padded message using AES
    iv = random.randbytes(16)  # Generate a random initialization vector (IV)
    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_message = encryptor.update(padded_data) + encryptor.finalize()

    return encrypted_message, iv

# Encrypt the AES key using the public key (RSA)
def encrypt_aes_key(public_key_pem, aes_key):
    public_key = serialization.load_pem_public_key(public_key_pem)
    encrypted_aes_key = public_key.encrypt(
        aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_aes_key

# Decrypt the AES key using the private key (RSA)
def decrypt_aes_key(private_key_pem, encrypted_aes_key):
    private_key = serialization.load_pem_private_key(private_key_pem, password=None)
    aes_key = private_key.decrypt(
        encrypted_aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return aes_key

# Decrypt the email using the AES key
def decrypt_email_aes(aes_key, encrypted_message, iv):
    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_padded_message = decryptor.update(encrypted_message) + decryptor.finalize()

    # Unpad the decrypted message
    unpadder = sym_padding.PKCS7(algorithms.AES.block_size).unpadder()
    decrypted_message = unpadder.update(decrypted_padded_message) + unpadder.finalize()

    return decrypted_message.decode('utf-8')

# Example usage
if __name__ == "__main__":
    # Step 1: Generate RSA Key Pair
    private_key_pem, public_key_pem = generate_key_pair()

    # Step 2: Generate a random AES key
    aes_key = generate_aes_key()

    # Step 3: Encrypt the email using the AES key
    email_message = "Hello, this is a secure email."
    encrypted_email, iv = encrypt_email_aes(aes_key, email_message)
    print("Encrypted Email (AES):", encrypted_email)

    # Step 4: Encrypt the AES key using RSA public key
    encrypted_aes_key = encrypt_aes_key(public_key_pem, aes_key)
    print("\nEncrypted AES Key (RSA):", encrypted_aes_key)

    # Step 5: Decrypt the AES key using RSA private key
    decrypted_aes_key = decrypt_aes_key(private_key_pem, encrypted_aes_key)

    # Step 6: Decrypt the email using the decrypted AES key
    decrypted_email = decrypt_email_aes(decrypted_aes_key, encrypted_email, iv)
    print("\nDecrypted Email:", decrypted_email)

Encrypted Email (AES): b'x\x90{G\x1c\xc2\xf4I\x06;\x06\xb8kf8\x8d\xae\x04\xca\xdb\xd1\x80\xe1\x8e\xe7BY\xbf\xc5|\xa5\xd1'

Encrypted AES Key (RSA): b'+\xfc\xb9\x1a\x07z\xe5.\xc2,\x0b\x13v\xfa}\x02\x06\x12\x87\x94\xc6\x0e}\xd4\t_X\xdf\xef\x90\xcc\'JyY.\x93\xd9\x9b\x8d\xecv\xd2\x04\xc8\x11\x1d-\xbbM\'\xfb\xd0P\xaa\xf6\x94\x82#\xcb2\xa5\xb5\xcawM\xce\x8e\xbf\xa3\x7f\xcd\xfa \x0f\x98\xdc\x86\xd0\x14\xcf~a:\xe9\x18\x00\x8b\xbb\r\xf4g"\x10 \xf7\xc6\xabu#\x1c\x1b_1I\xd3\xf3\xfd.\xd9h\xb71\xc7\xf2\xdb\xaf\xa9\xd7\x92\x05\x9b\xf9\x8e\xe4\xb18\x9f\xfd\x13\xeb\xe3c\xf2\x8c\x8a\x83\xa8\x1d\xb1vke\xa2h\xeeI\xa6\xe9\x95\xbb}\xd2\xf9\x15\x18\xfcN\x9eZ\xa3\x8b;\x16\xd7AT|\x97\xad1Jh\xf6Q\x10\xcd\x96m6\xe9\x93\xafD\xd9\xda\x11\xbea\xd5\xc0\xabq\xbf\xe5\x8d\x94\xf2\x12m(\xb2\xf8\xb5\xe1\x9f\xea\xef\xb2\x98\x7f\x8b\x9e\xa0\\\xe9\xe0\xc5A\xdd4-\x0cL\xec\xda\xe9\x9eXF\xef\x8e\x08\xc2\x8d7D\xda\xab\xaf\xad\xb9-\xc8\r\t\xdfw\xe7\xb6\xe9f\xf0\x9f\xbfr'

Decrypted Email: Hello, this is a secure email.


# Q2. Implement S/MIME to encrypt and sign the email messages.

In [ ]:
import smtplib
import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from smail import sign_message
import os
import sys

print("Starting the email message siginig and encryption.")

LOGIN_MAIL, LOGIN_PASS = open("LOGIN_INFO", "r").read().split("\n")

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

CERT_FILE = "certs/certificate.pem"
KEY_FILE = "certs/private_key.pem"

message = """
<h1>Hello User,</h1>
<br>
<hr>
<h3>I am a mail sent using python using with the help of smtplib, email and smail python libraries.</h3>
"""

msg = MIMEMultipart("related")
msg.attach(MIMEText(message, "html", _charset="UTF-8"))
msg["Subject"] = f"MESSAGE {datetime.datetime.today()}"
msg["From"] = LOGIN_MAIL
# msg["To"] = sys.argv[1]
msg["To"] = "gen.ai.keshav.2024@gmail.com"

signed_msg = sign_message(msg, KEY_FILE, CERT_FILE)
print("Message Signed Successfully")
with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login(LOGIN_MAIL, LOGIN_PASS)
    server.send_message(signed_msg)

with open("res.txt", 'w') as f:
    f.write(signed_msg.as_string())
    f.close()

print("SENT SUCCESSFULLY")

SENT SUCCESSFULLY


# Q3. Implement a Simple HTTPS Server Using LTS

In [ ]:
import http.server
import ssl

# Define the server address and port
server_address = ('localhost', 1110)  # Using port 4443 for HTTPS

# Simple request handler
class SimpleHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        # Example: respond to GET requests
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(b"Hello, this is a secure HTTPS server!")

# Create the HTTP server
httpd = http.server.HTTPServer(server_address, SimpleHTTPRequestHandler)

# Configure SSL context
context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
context.load_cert_chain(certfile="server.pem", keyfile="server.pem")

# Wrap the server's socket with the SSL context
httpd.socket = context.wrap_socket(httpd.socket, server_side=True)

print(f"Serving on https://{server_address[0]}:{server_address[1]}")

# Run the server
httpd.serve_forever()

# Q4. Implement secure communication between a client and server using TLS.

#### tls_sserver.py

In [ ]:
import socket
import ssl

# Define server address and port
server_address = ('localhost', 4443)

# Set up SSL context for the server
context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
context.load_cert_chain(certfile="server.pem")  # Load server certificate and private key

# Create a TCP socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0) as sock:
    sock.bind(server_address)
    sock.listen(5)
    print("Server listening on", server_address)

    # Wrap the socket with the SSL context for secure communication
    with context.wrap_socket(sock, server_side=True) as ssock:
        # Accept client connections
        while True:
            client_socket, client_address = ssock.accept()
            print("Client connected from", client_address)
            try:
                # Receive data from the client
                data = client_socket.recv(1024)
                print("Received:", data.decode())
                
                # Send a response to the client
                client_socket.sendall(b"Hello, client. This is a secure response.")
            except Exception as e:
                print("Error:", e)
            finally:
                client_socket.close()

Server listening on ('localhost', 1000)


#### tls_client.py

In [ ]:
import socket
import ssl

# Define server address and port
server_address = ('localhost', 4443)

# Set up SSL context for the client
context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
context.load_verify_locations('server.pem')  # Load server certificate to verify

# ** Ignore hostname checking ** (for testing purposes)
context.check_hostname = False

# Create a TCP socket
with socket.create_connection(server_address) as sock:
    # Wrap the socket with SSL context
    with context.wrap_socket(sock, server_hostname='localhost') as ssock:
        # Send data to the server
        ssock.sendall(b"Hello, secure server!")
        
        # Receive response from the server
        data = ssock.recv(1024)
        print("Received:", data.decode())
